In [1]:
import pandas as pd
import numpy as np
import datetime
import os
from haversine import haversine

os.getcwd()

'/home/jian/Projects/Savers/QC_CA_TA_zips_Pctg'

In [2]:
Excel_1=pd.ExcelFile("/home/jian/Projects/Savers/QC_CA_TA_zips_Pctg/Savers_CA_TA_Final_Version_JL_2019-03-27 BR.xlsx")
print(Excel_1.sheet_names)
df_1=Excel_1.parse("TA_zips",dtype=str,usecols=['location_id','zip_cd'])
print(df_1.shape)
df_1=df_1.drop_duplicates()
print(df_1.shape)

['Store_list', 'TA_Summary', 'TA_Summary_by_store_count', 'TA_zips', 'CAN TA_zips De-Duped', 'check Dup rate']
(981422, 2)
(981422, 2)


In [3]:
df_1.head(2)

,location_id,zip_cd
0,2004,V1Y 8C8
1,2004,V1Y 5T5


In [4]:
df_1_store_center=Excel_1.parse("Store_list",dtype=str,usecols=['location_id','Latitude','Longitude'])
df_1_store_center.shape

(143, 3)

In [5]:
df_2_1=pd.read_excel("./Savers_zips_for_2156_RL_2019-06-28.xlsx",dtype=str)
df_2_2=pd.read_excel("./Savers_zips_for_2161_RL_2019-06-28.xlsx",dtype=str)
df_2_2['store_id']="2161" # Correct the values in teh column for the store number
df_2=df_2_1.append(df_2_2)
df_2['Google_Lat']=df_2['Google_Lat'].astype(float)
df_2['Google_Long']=df_2['Google_Long'].astype(float)
df_2=df_2[['store_id','Google_Lat','Google_Long','zip_cd']]
df_2=df_2.rename(columns={"Google_Lat":"Latitude","Google_Long":"Longitude"})
df_2=df_2.rename(columns={"store_id":"location_id"})


In [6]:
df_3=pd.ExcelFile("./Savers_New2Stores_TA 20191017 CC.xlsx")
df_3=df_3.parse("All Data",dtype=str)
df_3['latitude_meas']=df_3['latitude_meas'].astype(float)
df_3['longitude_meas']=df_3['longitude_meas'].astype(float)
df_3=df_3[['location_id','latitude_meas','longitude_meas','zip','Zip first 3']]
df_3=df_3.rename(columns={"latitude_meas":"Latitude","longitude_meas":"Longitude"})
df_3=df_3.rename(columns={"zip":"zip_cd"})


In [7]:

df_2_store_center=df_2[['location_id','Latitude','Longitude']].drop_duplicates()
df_3_store_center=df_3[['location_id','Latitude','Longitude']].drop_duplicates()
# 
df_1_store_center['store_center']=df_1_store_center[['Latitude','Longitude']].values.tolist()
df_2_store_center['store_center']=df_2_store_center[['Latitude','Longitude']].values.tolist()
df_3_store_center['store_center']=df_3_store_center[['Latitude','Longitude']].values.tolist()
# 

df_store_centers=df_1_store_center.append(df_2_store_center).append(df_3_store_center)

dict_CA_store_centers=df_store_centers.set_index("location_id").to_dict()['store_center']

In [8]:
len(dict_CA_store_centers)

147

In [9]:
df_1=df_1[['location_id','zip_cd']].drop_duplicates()
df_2=df_2[['location_id','zip_cd']].drop_duplicates()
df_3=df_3[['location_id','zip_cd']].drop_duplicates()

df_all_zips_by_store=df_1.append(df_2).append(df_3)

In [10]:
print(df_all_zips_by_store.shape)
df_all_zips_by_store=df_all_zips_by_store.drop_duplicates()
print(df_all_zips_by_store.shape)

(1006010, 2)
(1006010, 2)


In [11]:
df_unique_zip_6=df_all_zips_by_store[['zip_cd']].drop_duplicates()
df_unique_zip_6.shape

(562889, 1)

In [12]:
df_unique_zip_6.head(2)

,zip_cd
0,V1Y 8C8
1,V1Y 5T5


In [13]:
df_CA_zip_data=pd.read_csv("/home/jian/Docs/Geo_mapping/Canadian_Postal_Codes/canada-postal-code-database-BUSINESS-CSV/canada-postal-code-database-BUSINESS.csv",
                           dtype=str,usecols=['PostalCode','Latitude','Longitude'])
print(df_CA_zip_data.shape)
df_CA_zip_data=df_CA_zip_data.drop_duplicates()
print(df_CA_zip_data.shape)
print(df_CA_zip_data['PostalCode'].nunique())

(1115041, 3)
(873339, 3)
873339


In [14]:
df_CA_zip_data['zip_3']=df_CA_zip_data['PostalCode'].apply(lambda x: x.split(" ")[0])
print(df_CA_zip_data['zip_3'].apply(len).unique())

[3]


In [16]:
df_zip3_zip6_count=df_CA_zip_data.groupby("zip_3")['PostalCode'].count().to_frame().reset_index().rename(columns={"PostalCode":"zip6_total_count"})
print(df_zip3_zip6_count.shape)



(1653, 2)


In [17]:
df_unique_zip_6['zip_3']=df_unique_zip_6['zip_cd'].apply(lambda x: x.split(" ")[0])
print(df_unique_zip_6['zip_3'].apply(len).unique())

list_all_TA_zips_3=df_unique_zip_6['zip_3'].unique().tolist()

[3]


In [18]:
df_included_zip6_count=df_unique_zip_6.groupby("zip_3")['zip_cd'].count().to_frame().reset_index().rename(columns={"zip_cd":"zip6_in_TA"})
df_zip3_zip6_count.shape

(1653, 2)

In [19]:
df_output_1_overall=pd.merge(df_zip3_zip6_count,df_included_zip6_count,on="zip_3",how="outer")
df_output_1_overall=df_output_1_overall.fillna(0)

df_output_1_overall['Pctg_in_TA']=df_output_1_overall['zip6_in_TA']/df_output_1_overall['zip6_total_count']
df_output_1_overall['Pctg_in_TA']=df_output_1_overall['Pctg_in_TA'].apply(lambda x: np.round(x,6))

In [20]:
df_output_1_overall['Label_in_TA']=np.where(df_output_1_overall['zip_3'].isin(list_all_TA_zips_3),1,0)

In [21]:
df_output_2_by_store=df_all_zips_by_store.copy()
df_output_2_by_store['zip_3']=df_output_2_by_store['zip_cd'].apply(lambda x: x.split(" ")[0])
print(df_output_2_by_store['zip_3'].apply(len).unique())
df_output_2_by_store=df_output_2_by_store.groupby(["location_id","zip_3"])['zip_cd'].count().to_frame().reset_index().rename(columns={"zip_cd":"zip6_in_TA_by_store"})
df_output_2_by_store=pd.merge(df_output_2_by_store,df_zip3_zip6_count,on="zip_3",how="left")

df_output_2_by_store['Pctg_in_TA_store']=df_output_2_by_store['zip6_in_TA_by_store']/df_output_2_by_store['zip6_total_count']
df_output_2_by_store['Pctg_in_TA_store']=df_output_2_by_store['Pctg_in_TA_store'].apply(lambda x: np.round(x,6))

[3]


In [22]:
writer=pd.ExcelWriter("./Savers_QC_zip6_to_zip3_converting_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_1_overall.to_excel(writer,"unique_zip3_whole",index=False)
df_output_2_by_store.to_excel(writer,"zip3_by_store",index=False)
writer.save()